Dont know what i think so far. I am adding more parameters, but now the can learn different non-linearities. Adding more parameters will lead to over fitting? But i can just reduce the nets size?

I could tie weights across
* neurons?
* read/write????

Based on 
* Balduzzi - Strongly typed RNNs
* and Neural Turing Machines.


Permutations
* Additive, multiplication
* Walks through weight space, aka the environment

Analogy
* Weights = position in environment
* Memory = environment
* Neurons = ants
* ???

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Hyper parameters
inputs = 28
hidden = 0
outputs = 10
m = inputs + hidden + outputs #number of memory cells

n = 10#number of neurons

In [50]:
#Neurons that dont have a state
class Neurons():
    def __init__(self,n):
        self.alphas = tf.Variable(np.ones((n,1)),'alpha')
        self.betas = tf.Variable(np.ones((n,1)),'beta')
        
    def forward(self,phi):
        return  tf.exp(phi*self.betas) *self.alphas 

In [56]:
x = tf.placeholder(tf.float64,shape = [inputs,1])
y = tf.placeholder(tf.float64, shape = [outputs])

#The environment where signals interact.
memory = tf.Variable(np.ones((m,1)),'Memory') 

#The weights. Do we want these to be symmetric? Do we want to learn these?
read = tf.Variable(np.random.standard_normal((n,m)),'ReadConnectionWeights')
write = tf.Variable(np.random.standard_normal((m,n)),'WriteConnectionWeights')
biases = tf.Variable(np.ones((n,1)),'Biases')

neurons = Neurons(n)

In [57]:
# Inputs
i = tf.slice(memory, [0,0],[inputs,1])
j = tf.slice(memory, [inputs,0], [m-inputs,1])
i = i * x
mem = tf.concat(0,[i,j])

In [58]:
### Dynamics
#the inputs to our neurons. 
phi = tf.matmul( read , mem ) + biases #nxm x mx1 = nx1
#the inputs to our memory cells. 
psi = tf.matmul( write , neurons.forward(phi) ) #mxn x nx1 = mx1

memory = psi*memory #can i do this? will gradients work?

In [59]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [60]:
ans = sess.run([mem,i,j,phi,psi,memory],feed_dict={x : 100*np.random.random((28,1)) })
for a in ans:
    print(a.shape)

(38, 1)
(28, 1)
(10, 1)
(10, 1)
(38, 1)
(38, 1)
